In [1]:
from ultralytics import YOLO
import os
from PIL import Image

# Load YOLOv8 pretrained on COCO
model = YOLO("yolov8m.pt")  # You can also try 'yolov8n.pt' or 'yolov8x.pt'

# Define dataset root path
dataset_path = r"C:\Users\itzdh\Desktop\Object Detection\Dataset\images"

# Class mapping: folder name ➝ custom ID + COCO ID
class_map = {
    "Phone": {"coco_id": 67, "custom_id": 0},
    "Mouse": {"coco_id": 65, "custom_id": 1},
    "Plastic_Bottles": {"coco_id": 39, "custom_id": 2},
    "Steel_Bottles": {"coco_id": 39, "custom_id": 3}
}

# Go through each class directory
for folder, ids in class_map.items():
    image_dir = os.path.join(dataset_path, folder)
    label_dir = os.path.join(image_dir, "labels")
    os.makedirs(label_dir, exist_ok=True)

    for filename in os.listdir(image_dir):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(image_dir, filename)

        # Verify image is readable
        try:
            with Image.open(image_path) as im:
                im.verify()
        except Exception:
            print(f"Skipped invalid/corrupted image: {filename}")
            continue

        # Run inference
        try:
            result = model(image_path)[0]
        except Exception:
            print(f"Model failed to process: {filename}")
            continue

        labels = []
        for box in result.boxes:
            detected_id = int(box.cls[0].item())
            if detected_id != ids["coco_id"]:
                continue

            x, y, w, h = box.xywhn[0].tolist()
            labels.append(f"{ids['custom_id']} {x:.6f} {y:.6f} {w:.6f} {h:.6f}")

        # Save labels to YOLO txt format
        label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
        with open(label_path, "w") as f:
            f.write("\n".join(labels))

        print(f"Labeled: {folder}/{filename}")

print("Auto-labeling complete for all folders.")


c:\Users\itzdh\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(



image 1/1 C:\Users\itzdh\Desktop\Object Detection\Dataset\images\Phone\phone_001.jpg: 608x640 (no detections), 412.6ms
Speed: 4.6ms preprocess, 412.6ms inference, 0.9ms postprocess per image at shape (1, 3, 608, 640)
Labeled: Phone/phone_001.jpg

image 1/1 C:\Users\itzdh\Desktop\Object Detection\Dataset\images\Phone\phone_002.jpg: 512x640 1 cell phone, 348.0ms
Speed: 2.9ms preprocess, 348.0ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)
Labeled: Phone/phone_002.jpg

image 1/1 C:\Users\itzdh\Desktop\Object Detection\Dataset\images\Phone\phone_003.jpg: 448x640 1 cell phone, 372.2ms
Speed: 1.8ms preprocess, 372.2ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
Labeled: Phone/phone_003.jpg

image 1/1 C:\Users\itzdh\Desktop\Object Detection\Dataset\images\Phone\phone_004.jpg: 640x640 2 cars, 1 cell phone, 1 scissors, 411.1ms
Speed: 5.5ms preprocess, 411.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Labeled: Phone/phone_004.jpg

